In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
print(os.listdir("../input"))

In this kernel I just load the competition data, clean it using pre-stored information from the "Springleaf competition : EDA" kernel and save the results. The results are to be used in script that build different classificaiton models.

In [ ]:
train_df = pd.read_csv('../input/springleaf-marketing-response/train.csv')
test_df = pd.read_csv('../input/springleaf-marketing-response//test.csv')
train_orig_shape = train_df.shape
test_orig_shape = test_df.shape

In [ ]:
data = [train_df, test_df]

In [ ]:
prep_data = np.load('../input/springleaf-competition-eda/prep_info.npz')
missing_to_remove, missing_to_fill, missing_example_to_remove, constants, duplicates, dates, potential_to_remove = \
    prep_data["arr_0"], prep_data["arr_1"], prep_data["arr_2"], prep_data["arr_3"], prep_data["arr_4"], prep_data["arr_5"], prep_data["arr_7"]
type(potential_to_remove)

Clearing features in missing_to_remove, constants and duplicates

In [ ]:
for df in data:
    df.drop(np.concatenate((missing_to_remove, constants, duplicates)), axis=1, inplace=True)
data[0].shape

Dealing with the rest of missing data:

In [ ]:
# updating missing_to_fill and missing_to_remove
missing_to_fill = [feat for feat in missing_to_fill if feat not in np.concatenate((missing_to_remove, constants, duplicates))]

# separating categorical and numerical features since these are treated differently
categorical = train_df.select_dtypes(include=[np.object]).columns
numerical = train_df.select_dtypes(include=[np.number]).columns

# filling in the missing data for features containing many of them
for df in data:
    for col in missing_to_fill:
        try:
            if col in categorical:
                # fill missing data with mode
                df[col].fillna(df[col].mode()[0], inplace=True)
            else:
                # fill missing data with mean
                df[col].fillna(df[col].mean(), inplace=True)
        except KeyError:
            print("Feature {} has already been removed!".format(col))

In [ ]:
# removing rows with missing data in the training data, but filling those in test data
data[0].dropna(axis=0, inplace=True)
missing_examples_to_remove = [feat for feat in data[1].columns if data[1][feat].isnull().sum() > 0]
for col in missing_examples_to_remove:
    try:
        if col in categorical:
            # fill missing data with mode
            test_df[col].fillna(data[0][col].mode()[0], inplace=True)
        else:
            # fill missing data with mean
            test_df[col].fillna(data[0][col].mean(), inplace=True)
    except KeyError:
        print("Feature {} has already been removed!".format(col))

In [ ]:
# making sure there are no more missing entries
train_cleared_shape = data[0].shape
test_cleared_shape = data[1].shape
data[0].isnull().sum().max(), data[1].isnull().sum().max()

In [ ]:
train_cleared_shape, test_cleared_shape

Encoding categorical data, for now we use simple integer encoding instead of one-hot encoding, we may need to come back to this later

In [ ]:
from datetime import datetime
year_func = lambda x: datetime.strptime(x, "%d%b%y:%H:%M:%S" ).year
month_func = lambda x: datetime.strptime(x, "%d%b%y:%H:%M:%S" ).month
day_func = lambda x: datetime.strptime(x, "%d%b%y:%H:%M:%S" ).day
hour_func = lambda x: datetime.strptime(x, "%d%b%y:%H:%M:%S" ).hour
minute_func = lambda x: datetime.strptime(x, "%d%b%y:%H:%M:%S" ).minute

# we perform the encoding inplace because of limited RAM
for i in range(len(data)):
    for col in categorical:
        if col in dates:
            data[i][col + '_year'] = data[i][col].map(year_func)
            data[i][col + '_month'] = data[i][col].map(month_func)
            data[i][col + '_day'] = data[i][col].map(day_func)
            data[i][col + '_hour'] = data[i][col].map(hour_func)
            data[i][col + '_minute'] = data[i][col].map(minute_func)
        data[i][col] = data[i][col].factorize()[0]
train_encod_shape = data[0].shape
test_encod_shape = data[1].shape
data[0].head(20)

In [ ]:
train_encod_shape, test_encod_shape

Saving data, also saving a smaller chunk of it for rapid prototyping of models

In [ ]:
data[0].to_csv('train_clean.csv')
data[1].to_csv('test_clean.csv')
data[0].sample(frac=0.2).to_csv('train_clean_small.csv')
data[1].sample(frac=0.2).to_csv('test_clean_small.csv')

In [ ]:
print("Shape of training data:")
print("Original : {}    After cleaning: {}    After encoding: {}".format(train_orig_shape, train_cleared_shape, train_encod_shape))
print("Shape of test data:")
print("Original : {}    After cleaning: {}    After encoding: {}".format(test_orig_shape, test_cleared_shape, test_encod_shape))